In [9]:
import pyshark
import pandas as pd
import sys

In [19]:
capture = pyshark.FileCapture(r"C:\Users\user\OneDrive - mail.tau.ac.il\personal\os_classifier\data_cleaning\nmap_300_pcap.pcapng")

In [ ]:
tcp_inbound = list(filter(lambda p : p.ip.dst == '192.168.1.12' and 'tcp' in p, capture))

In [ ]:
def get_opts_order(p):
    if not hasattr(p.tcp, "options"):
        return ''
    i = 0
    opt_split = p.tcp.options.split(":")
    opt_order = []
    while i < len(opt_split):
        curr_opt = opt_split[i]
        opt_order.append(curr_opt)
        i += 1
        if curr_opt == '01' or curr_opt == '00':
            continue
        else:
            # read option's length field & jump to the next option
            i += int(opt_split[i], 16) - 1
    return '-'.join(opt_order)

In [ ]:
def check_nonzero(p):
    # if there's a Timestamp & ACK is not set, make sure TSecr is 0
    if  hasattr(p.tcp, "options") and (not p.tcp.flags_ack.base16_value):
        pass        

In [ ]:
data = []

for p in tcp_inbound:
    p_data = {}
    p_data['ttl'] = p.ip.ttl.base16_value
    p_data['df']  = p.ip.flags_df.base16_value
    p_data['mss'] = -1
    if hasattr(p.tcp, 'options_mss_val'):
        p_data['mss'] = p.tcp.options_mss_val.base16_value
    p_data['w_size'] = p.tcp.window_size.base16_value
    p_data['w_scale'] = 1
    if hasattr(p.tcp, 'options_wscale_shift'):
        p_data['w_scale'] = p.tcp.options_wscale_shift.base16_value
    p_data['opts_order'] = get_opts_order(p)
    p_data['df+'] = 1 if (p.ip.flags_df.base16_value and not p.ip.id.base16_value) else 0
    p_data['df-'] = 1 if (not p.ip.flags_df.base16_value and p.ip.id.base16_value) else 0
    p_data['fo+'] = 1 if (not p.ip.flags_df.base16_value and p.ip.frag_offset.base16_value) else 0
    p_data['fo-'] = 1 if (p.ip.flags_df.base16_value and not p.ip.frag_offset.base16_value) else 0
    p_data['ecn'] = p.ip.dsfield_ecn.base16_value
    p_data['seq0'] = 1 if (p.tcp.seq_raw.base16_value == 0) else 0
    data.append(p_data)

In [ ]:
pd.DataFrame.from_dict(data).to_csv(sys.argv[1])